In [1]:
# Import the necessary libraries and install packages
!pip install -q git+https://github.com/openai/whisper.git > /dev/null

import shutil,os
import subprocess
import random,torch
import whisper

torch.manual_seed(42)
random.seed(42)

In [2]:
model_size = "medium"     # ['tiny', 'base', 'small', 'medium', 'large']
language = "English"      # ['any', 'English']

model_name = model_size
if language == 'English' and model_size != 'medium':
  model_name += '.en'

model = whisper.load_model(model_size)

100%|██████████████████████████████████████| 1.42G/1.42G [00:12<00:00, 120MiB/s]


In [3]:
# upload audio file
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def generate_transcript(path):

  shutil.copy(path,os.getcwd())
  path = path.split("/")[-1]
  filename = path[:-4].split(' ')
  if len(filename)==1:
    filename = filename[0]
  else:
    filename=f"{filename[0]}{filename[1]}"
  
  if path[-3:] != 'wav':
    subprocess.call(['ffmpeg', '-i', path, 'audio.wav', '-y'])
    os.remove(path)
    path = 'audio.wav'

  print("File conversion - Done,Now Transcribing")

  result = model.transcribe(path)
  segments = result["segments"]

  f = open(f"{filename}.txt", "w")

  text = ""

  # Iterate through the segments and write the transcribed text to the text file
  for (i, segment) in enumerate(segments):
      try:
        f.write(segment["text"][1:] + '\n\n') 
      except:
        continue
  f.close()

  print(f"Transcription - Done,check {filename}.txt")


In [5]:

path = "/content/drive/MyDrive/Audio/Audio G.wav"

generate_transcript(path)

File conversion - Done,Now Transcribing
Transcription - Done,check AudioG.txt
